<a href="https://colab.research.google.com/github/SatyaKuppam/QUBOs-and-Ising-models/blob/master/covering_and_packing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Covering and packing problems:

* Exact cover

In [0]:
!pip install dwave-ocean-sdk

In [0]:
import networkx as nx
import dwave_networkx as dnx
import matplotlib
import matplotlib.pyplot as plt
import dimod
%matplotlib inline

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.cloud import Client
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave_qbsolv import QBSolv
from collections import defaultdict

import networkx as nx
import dwave_networkx as dnx
import neal

## Exact Cover

*Definition*: An instance $(U, V)$ of a set covering problem consists of a finite set $U$ and a family $H$ of subsets of $U$, such that every element of $U$ belongs to at least one subset in $V$. (From [1]).

* The subet $C_i \in V$ *covers* the elements of $U$. $$V = \{C_1...C_N\}$$ 
$$U = \bigcup\limits_{i=1}^{N} C_i$$  

* The problem is to find the minimum size subset $C \subseteq H$ whose members cover all elements of $U$. $$U = \bigcup\limits_{C_i \in C} C_i$$ This is $NP$-hard.

* The decision version of this problem is $NP$-complete. In the decision cersion we are asked if a covering exists with size atmost $k$.

### Decision version.

The energy functions for the decision version and the $NP$-hard version are derived from *Lucas*[2].

First the decision version: $H = H_A  + H_B$. $H_A$ ensures that we have an exact cover and $H_B$ ensure that the cover is atmost $k$ which the user provides.

$$H_A = \sum_{\alpha \in U} A(1 - \sum_{\alpha \in V_i} x_i)^2=\sum_{\alpha \in U} A(1 + \sum_{\alpha \in V_i}x_i \sum_{\alpha \in V_j}x_j - 2\sum_{\alpha \in V_i}x_i)$$

$$H_B = B (\sum_{i=1}^{|V|}x_i-k)$$

So we atleast need $|V|$ spins to encode this problem.

In [0]:
class exact_cover:
  def __init__(self, U, V):
    self.U = U
    self.V = V
    self.vartype = dimod.BINARY
    self.linear = defaultdict(int)
    self.quadratic = defaultdict(int)
    self.offset = 0.0
    
    self.n = len(self.U)
    self.B = 1.0
    self.A = self.n * self.B + 1.0
    print(f"Setting A:{self.A} and B:{self.B}")
    self.N = len(self.V)

    if self.n == 0 or self.N == 0:
      raise ValueError("U or V cannot be empty.")

    self.solver = None

  def _energy_function_h_a(self):
    self.offset += self.A

    for u in self.U:
      for idx_i, v_i in enumerate(self.V):
        if u in v_i:
          for idx_j, v_j in enumerate(self.V):
            if u in v_j:
              if idx_i == idx_j:
                self.linear[idx_i] += self.A
              else:
                self.quadratic[(idx_i, idx_j)] += self.A
    
    for u in self.U:
      for idx, v_i in enumerate(self.V):
        if u in v_i:
          self.linear[idx] -= 2 * self.A

  def _energy_function_h_b(self, k=0.0):
    self.offset -= self.B * k

    for u in U:
      for idx, v_i in enumerate(V):
        self.linear[idx] += self.B

  def decide(self, k, solver=None, **kwargs):
    self._energy_function_h_a()
    self._energy_function_h_b(k)
    return self._solve(solver=solver, **kwargs)

  def solve(self, solver=None, **kwargs):
    self._energy_function_h_a()
    self._energy_function_h_b()
    return self._solve(solver=solver, **kwargs)

  def _solve(self, solver=None, **kwargs):
    if solver == None:
      print("Setting the solver as QBsolv")
      solver = QBSolv()

    print(self.linear)
    print(self.quadratic)
    print(self.offset)
    bqm = dimod.BinaryQuadraticModel(self.linear, self.quadratic, self.offset,
                                     self.vartype)
    return solver.sample(bqm, **kwargs)
  

Testing $H_A$, all covers should be equally likely.

In [6]:
U = [1,2,3,4,5,6]
V = [[1], [2], [3],[4],[5],[6],[1,2],[3,4],[5,6]]

exact_cover_decision_h_a = exact_cover(U, V)
exact_cover_decision_h_a._energy_function_h_a()
exact_cover_decision_h_a._solve(num_repeats=1000)

Setting A:7.0 and B:1.0
Setting the solver as QBsolv
defaultdict(<class 'int'>, {0: -7.0, 6: -14.0, 1: -7.0, 2: -7.0, 7: -14.0, 3: -7.0, 4: -7.0, 8: -14.0, 5: -7.0})
defaultdict(<class 'int'>, {(0, 6): 7.0, (6, 0): 7.0, (1, 6): 7.0, (6, 1): 7.0, (2, 7): 7.0, (7, 2): 7.0, (3, 7): 7.0, (7, 3): 7.0, (4, 8): 7.0, (8, 4): 7.0, (5, 8): 7.0, (8, 5): 7.0})
7.0


SampleSet(rec.array([([0, 0, 1, 1, 1, 1, 1, 0, 0], -35., 132),
           ([0, 0, 0, 0, 0, 0, 1, 1, 1], -35., 119),
           ([0, 0, 1, 1, 0, 0, 1, 0, 1], -35., 124),
           ([0, 0, 0, 0, 1, 1, 1, 1, 0], -35., 125),
           ([1, 1, 0, 0, 0, 0, 0, 1, 1], -35., 129),
           ([1, 1, 1, 1, 0, 0, 0, 0, 1], -35., 128),
           ([1, 1, 0, 0, 1, 1, 0, 1, 0], -35., 110),
           ([1, 1, 1, 1, 1, 1, 0, 0, 0], -35., 134)],
          dtype=[('sample', 'i1', (9,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8], {}, 'BINARY')

Testing $H_B$, no subset should be selected because the minimum energy possible is $-Bk$.

In [91]:
exact_cover_decision_h_b = exact_cover(U, V)
exact_cover_decision_h_b._energy_function_h_b(k=3)
exact_cover_decision_h_b._solve(num_repeats=1000)

Setting A:7.0 and B:1.0
Setting the solver as QBsolv
defaultdict(<class 'int'>, {0: 6.0, 1: 6.0, 2: 6.0, 3: 6.0, 4: 6.0, 5: 6.0, 6: 6.0, 7: 6.0, 8: 6.0})
defaultdict(<class 'int'>, {})
-3.0


SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 0, 0, 0], -3., 1001)],
          dtype=[('sample', 'i1', (9,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8], {}, 'BINARY')

$C=[[1,2],[3,4],[5,6]]$ should be selected with high probability.

In [7]:
exact_cover_decision = exact_cover(U, V)
exact_cover_decision.decide(k=3, num_repeats=1000)

Setting A:7.0 and B:1.0
Setting the solver as QBsolv
defaultdict(<class 'int'>, {0: -1.0, 6: -8.0, 1: -1.0, 2: -1.0, 7: -8.0, 3: -1.0, 4: -1.0, 8: -8.0, 5: -1.0})
defaultdict(<class 'int'>, {(0, 6): 7.0, (6, 0): 7.0, (1, 6): 7.0, (6, 1): 7.0, (2, 7): 7.0, (7, 2): 7.0, (3, 7): 7.0, (7, 3): 7.0, (4, 8): 7.0, (8, 4): 7.0, (5, 8): 7.0, (8, 5): 7.0})
4.0


SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 1, 1, 1], -20., 605),
           ([0, 0, 1, 1, 0, 0, 1, 0, 1], -14., 160),
           ([0, 0, 0, 0, 1, 1, 1, 1, 0], -14., 103),
           ([1, 1, 0, 0, 0, 0, 0, 1, 1], -14., 136)],
          dtype=[('sample', 'i1', (9,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8], {}, 'BINARY')

No solution exists with $k=2$.

In [8]:
exact_cover_decision = exact_cover(U, V)
exact_cover_decision.decide(k=2, num_repeats=1000)

Setting A:7.0 and B:1.0
Setting the solver as QBsolv
defaultdict(<class 'int'>, {0: -1.0, 6: -8.0, 1: -1.0, 2: -1.0, 7: -8.0, 3: -1.0, 4: -1.0, 8: -8.0, 5: -1.0})
defaultdict(<class 'int'>, {(0, 6): 7.0, (6, 0): 7.0, (1, 6): 7.0, (6, 1): 7.0, (2, 7): 7.0, (7, 2): 7.0, (3, 7): 7.0, (7, 3): 7.0, (4, 8): 7.0, (8, 4): 7.0, (5, 8): 7.0, (8, 5): 7.0})
5.0


SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 1, 1, 1], -19., 604),
           ([1, 1, 0, 0, 0, 0, 0, 1, 1], -13., 157),
           ([0, 0, 0, 0, 1, 1, 1, 1, 0], -13., 109),
           ([0, 0, 1, 1, 0, 0, 1, 0, 1], -13., 134)],
          dtype=[('sample', 'i1', (9,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8], {}, 'BINARY')

For $k=4$, we observe that its not the ground state, although solution is possible.

In [9]:
exact_cover_decision = exact_cover(U, V)
exact_cover_decision.decide(k=4, num_repeats=1000)

Setting A:7.0 and B:1.0
Setting the solver as QBsolv
defaultdict(<class 'int'>, {0: -1.0, 6: -8.0, 1: -1.0, 2: -1.0, 7: -8.0, 3: -1.0, 4: -1.0, 8: -8.0, 5: -1.0})
defaultdict(<class 'int'>, {(0, 6): 7.0, (6, 0): 7.0, (1, 6): 7.0, (6, 1): 7.0, (2, 7): 7.0, (7, 2): 7.0, (3, 7): 7.0, (7, 3): 7.0, (4, 8): 7.0, (8, 4): 7.0, (5, 8): 7.0, (8, 5): 7.0})
3.0


SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 1, 1, 1], -21., 591),
           ([0, 0, 0, 0, 1, 1, 1, 1, 0], -15., 100),
           ([0, 0, 1, 1, 0, 0, 1, 0, 1], -15., 171),
           ([1, 1, 0, 0, 0, 0, 0, 1, 1], -15., 142)],
          dtype=[('sample', 'i1', (9,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8], {}, 'BINARY')

We will find the smallest exact cover. The solution should choose $C=[[1,2],[3,4],[5,6]]$ with high probability.

In [10]:
exact_cover_find = exact_cover(U, V)
exact_cover_find.solve(num_repeats=1000)

Setting A:7.0 and B:1.0
Setting the solver as QBsolv
defaultdict(<class 'int'>, {0: -1.0, 6: -8.0, 1: -1.0, 2: -1.0, 7: -8.0, 3: -1.0, 4: -1.0, 8: -8.0, 5: -1.0})
defaultdict(<class 'int'>, {(0, 6): 7.0, (6, 0): 7.0, (1, 6): 7.0, (6, 1): 7.0, (2, 7): 7.0, (7, 2): 7.0, (3, 7): 7.0, (7, 3): 7.0, (4, 8): 7.0, (8, 4): 7.0, (5, 8): 7.0, (8, 5): 7.0})
7.0


SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 1, 1, 1], -17., 582),
           ([1, 1, 0, 0, 0, 0, 0, 1, 1], -11., 128),
           ([0, 0, 0, 0, 1, 1, 1, 1, 0], -11.,  96),
           ([0, 0, 1, 1, 0, 0, 1, 0, 1], -11., 198)],
          dtype=[('sample', 'i1', (9,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3, 4, 5, 6, 7, 8], {}, 'BINARY')

### References:

[1] Cormen, Thomas H., Charles E. Leiserson, Ronald L. Rivest, and Clifford Stein. *Introduction to algorithms.* MIT press, 2009.

[2] Lucas, Andrew. "Ising formulations of many NP problems." *Frontiers in Physics 2* (2014): 5.
